## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-14-22-42-08 +0000,bbc,Musk's Grok signs $200m deal with Pentagon day...,https://www.bbc.com/news/articles/c628d9mre3go
1,2025-07-14-22-38-21 +0000,bbc,UK to start processing Syrian asylum claims again,https://www.bbc.com/news/articles/cy4y2e8neywo
2,2025-07-14-22-11-06 +0000,nyt,Tuesday Briefing: More Weapons for Ukraine,https://www.nytimes.com/2025/07/14/briefing/uk...
3,2025-07-14-22-05-32 +0000,bbc,Faroes-style tunnels could 'transform' fortune...,https://www.bbc.com/news/articles/cvg7jw27w1do
4,2025-07-14-22-00-26 +0000,bbc,"Post Office could be owned by its postmasters,...",https://www.bbc.com/news/articles/czjk2eynz1lo


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2313/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
30,trump,11
182,trade,4
129,fire,3
33,tariffs,3
16,weapons,3


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
31,2025-07-14-14-48-00 +0000,wsj,Trump is threatening to ramp up the U.S. trade...,https://www.wsj.com/economy/trade/european-uni...,40
51,2025-07-14-02-00-00 +0000,wsj,Chinese manufacturers looking to dodge steep U...,https://www.wsj.com/economy/trade/vietnam-chin...,38
5,2025-07-14-21-34-30 +0000,bbc,Trump threatens Russia with tariffs while unve...,https://www.bbc.com/news/articles/czdv20v9lp1o,25
13,2025-07-14-20-05-00 +0000,bbc,Trump weapons pledge marks major step forward ...,https://www.bbc.com/news/articles/cy4y2rv41pyo,22
52,2025-07-14-01-15-53 +0000,bbc,How Trump public broadcasting cuts could hit r...,https://www.bbc.com/news/articles/c20w51rkw4no,22


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
31,40,2025-07-14-14-48-00 +0000,wsj,Trump is threatening to ramp up the U.S. trade...,https://www.wsj.com/economy/trade/european-uni...
51,18,2025-07-14-02-00-00 +0000,wsj,Chinese manufacturers looking to dodge steep U...,https://www.wsj.com/economy/trade/vietnam-chin...
42,17,2025-07-14-09-30-00 +0000,wsj,Food banks across the country were already str...,https://www.wsj.com/economy/food-banks-are-run...
47,17,2025-07-14-06-08-00 +0000,wsj,"China’s exports grew at a faster clip in June,...",https://www.wsj.com/economy/trade/chinas-expor...
29,16,2025-07-14-14-55-07 +0000,bbc,"Boeing fuel switches safe, regulator says afte...",https://www.bbc.com/news/articles/ce9xpgnx3vdo
32,15,2025-07-14-14-31-32 +0000,bbc,Two women killed at Kentucky church as gunman ...,https://www.bbc.com/news/articles/cm2l5jn6254o
10,12,2025-07-14-20-35-00 +0000,bbc,Gaza father's outrage after Israeli strike kil...,https://www.bbc.com/news/articles/ckglpk9xjewo
17,12,2025-07-14-18-39-13 +0000,bbc,'Heart-breaking': Locals and visitors devastat...,https://www.bbc.com/news/articles/c4gd8d5jpy5o
25,12,2025-07-14-15-51-39 +0000,bbc,Nigeria declares public holiday to honour form...,https://www.bbc.com/news/articles/c93kxl08glwo
5,11,2025-07-14-21-34-30 +0000,bbc,Trump threatens Russia with tariffs while unve...,https://www.bbc.com/news/articles/czdv20v9lp1o


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
